Please install the necessary libraries using your terminal/shell in your env before running the code. You can check all the libraries mentioned in the next code block. To install any library you can use either one of the code mentioned below

pip install library_name

pip3 install library_name

In [1]:
import os
import glob
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import json
from matplotlib.backends.backend_pdf import PdfPages
import warnings
import numpy as np

warnings.filterwarnings('ignore')

class_name = { 0: 'Mass',
1: 'Spiculation',
2: 'Suspicious Calcification',
3: 'Architectural Distortion',
4: 'Asymmetry',
5: 'Focal Asymmetry',
6: 'Skin Thickening',
7: 'Global Asymmetry',
8: 'Suspicious Lymph Node',
9: 'Skin Retraction',
10: 'Nipple Retraction'
}

color_dict = {
    0: (255, 0, 0),      # Red
    1: (0, 255, 0),      # Green
    2: (0, 0, 255),      # Blue
    3: (255, 255, 0),    # Yellow
    4: (255, 165, 0),    # Orange
    5: (128, 0, 128),    # Purple
    6: (0, 255, 255),    # Cyan
    7: (255, 192, 203),  # Pink
    8: (128, 128, 0),    # Olive
    9: (0, 0, 0),        # Black
    10: (169, 169, 169)  # Dark Grey
}

output_labels_dir = "results/labels"  # Directory to save ground truth images
output_pred_dir = "results/predictions"  # Directory to save predicted images

progress_file = 'prediction_progress.json'

Mention dataset.yaml file path and run

In [2]:
path = "dataset.yaml"

def remove_cache_files(directory):
    cache_files = glob.glob(os.path.join(directory, "*.cache"))
    for cache_file in cache_files:
        os.remove(cache_file)
        print(f"Removed: {cache_file}")
        
with open(path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

Removed: /home/rshah133/bcd/dataset/val/labels.cache


Change the model and hyperparameters.

* Epochs = keep minimum 200
* imgsz = change acc to your model assigned
* patience = number of epochs model will check for accuracy improvement and then stop if accuracy is not improving
* set device = [0,1] if you have 2 GPUs requested and device = 0 if only 1 GPU/CPU
* save = True, this will help you save the model checkpoints on its own 
* save_period = 10, this is the number of epochs after which model will save the checkpoints
* resume = True, if kernel crashes, model will continue training from the previous checkpoint
* iou = keep iou between 0.5-0.7 and check for best metrics (hyperparameter to play with)
* optimzer  = AdamW is the best for our model from what I have noticed. you can experiment if you want
* learning rate, momentum, and weight decay are few more hyperparamerts you can play with

In [3]:
model = YOLO('yolov8s.yaml')
results = model.train(data = path, epochs = 200, imgsz = 1024, batch = 10, name = 'checkpoint', device = [0,1], patience = 25, save = True, save_period = 10, exist_ok = True, resume = True, iou = 0.5, optimizer = 'AdamW')

New https://pypi.org/project/ultralytics/8.3.70 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.68 🚀 Python-3.11.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81156MiB)
                                                      CUDA:1 (NVIDIA A100-SXM4-80GB, 81156MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=dataset.yaml, epochs=200, time=None, patience=25, batch=10, imgsz=1024, save=True, save_period=10, cache=False, device=[0, 1], workers=8, project=None, name=checkpoint, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

E0000 00:00:1738313344.629641 3424411 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738313344.684243 3424411 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

E0000 00:00:1738313404.571886 3424713 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738313404.576100 3424713 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorBoard: Start with 'tensorboard --logdir runs/detect/checkpoint', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=11
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


...

                   all         38        165      0.625     0.0309     0.0468      0.013

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    101/200      3.33G      2.556      2.885      2.259         21       1024: 100%|██████████| 34/34 [00:03<00:00, 10.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 16.45it/s]


                   all         38        165      0.488     0.0109     0.0107    0.00406
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 76, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

101 epochs completed in 0.130 hours.
Optimizer stripped from runs/detect/checkpoint/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/checkpoint/weights/best.pt, 22.6MB

Validating runs/detect/checkpoint/weights/best.pt...
Ultralytics 8.3.68 🚀 Python-3.11.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81156MiB)
                                                      CUDA:1 (NVIDIA A100-SXM4-80GB, 81156MiB)
YOLOv8s summary (fused): 168 layers, 11,129,841 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.64it/s]


                   all         38        165      0.629     0.0339      0.065     0.0299
                  Mass         18         27      0.209      0.185      0.151     0.0594
           Spiculation         30        109      0.239      0.128       0.12       0.03
Suspicious Calcification         12         16      0.472      0.059     0.0576     0.0164
Architectural Distortion          1          1          1          0          0          0
             Asymmetry          3          3          1          0      0.375      0.219
       Focal Asymmetry          2          2          1          0      0.012    0.00482
       Skin Thickening          1          1          1          0          0          0
      Global Asymmetry          1          1          1          0          0          0
 Suspicious Lymph Node          2          2          1          0          0          0
       Skin Retraction          2          2          0          0          0          0
     Nipple Retra

Implemented Tensorboard. Code will redirect you to a link which should be opened in **Google Chrome** if nothing is getting displayed on safari.  You can choose not to run this code block if you want to not see the dashboard and save time.

In [4]:
#!kill 1869322
%load_ext tensorboard
%tensorboard --logdir runs/detect/checkpoint

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/rshah133/.local/bin/tensorboard", line 5, in <module>
    from tensorboard.main import run_main
  File "/home/rshah133/.local/lib/python3.11/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/home/rshah133/.local/lib/python3.11/site-packages/tensorboard/default.py", line 30, in <module>
    import pkg_resources
  File "/home/rshah133/.local/lib/python3.11/site-packages/pkg_resources/__init__.py", line 90, in <module>
    from jaraco.text import drop_comment, join_continuation, yield_lines
  File "/home/rshah133/.local/lib/python3.11/site-packages/setuptools/_vendor/jaraco/text/__init__.py", line 12, in <module>
    from jaraco.context import ExceptionTrap
  File "/home/rshah133/.local/lib/python3.11/site-packages/setuptools/_vendor/jaraco/context.py", line 17, in <module>
    from backports import tarfile
ImportErro

Mention test_dataset.yaml file path and run

In [5]:
test_path = "test_dataset.yaml"

with open(test_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

Removed: /home/rshah133/bcd/dataset/train/labels.cache
Removed: /home/rshah133/bcd/dataset/test/labels.cache


Predicting on test data. Please delete the prediction_progress.json file everytime if you want to predict on the complete test data from the START or it will resume from where it stopped last (use this when you want to continue from where the kernel crashed)

In [6]:
def save_progress(current_index):
    with open(progress_file, 'w') as f:
        json.dump({'last_processed': current_index}, f)

def load_progress():
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)['last_processed']
    return 0

# Loading the best model
best_model = YOLO('runs/detect/checkpoint/weights/best.pt')

test_img_path = data_loaded['val']

start_index = load_progress()

chunk_size = 48  # Adjust based on your available memory
for i in range(start_index, len(test_img_path), chunk_size):
    chunk_end = min(i + chunk_size, len(test_img_path))
    current_chunk = test_img_path[i:chunk_end]
    
    try:
        results = best_model.predict(source = current_chunk, save = True, save_txt = True, conf = 0.10, batch = chunk_size//4, stream = True)
        
        # code for post - processing results for later
        for r in results:
            pass  
            
        save_progress(chunk_end)
        
    except Exception as e:
        save_progress(i)
        print(f"Prediction stopped at image {i}. Progress saved.")
        raise e



image 1/706 /home/rshah133/bcd/dataset/test/images/002460132586dc0c7b88a59dce6e77bd.jpg: 1024x1024 1 Mass, 2.8ms
image 2/706 /home/rshah133/bcd/dataset/test/images/008bc6050f6d31fc255e5d87bcc87ba2.jpg: 1024x1024 (no detections), 2.8ms
image 3/706 /home/rshah133/bcd/dataset/test/images/008c66563c73b2f5b8e42915b2cd6af5.jpg: 1024x1024 (no detections), 2.8ms
image 4/706 /home/rshah133/bcd/dataset/test/images/00a6b0d56eb5136c1be2c3d624b04dad.jpg: 1024x1024 (no detections), 2.8ms
image 5/706 /home/rshah133/bcd/dataset/test/images/01df962b078e38500bf9dd9969a50083.jpg: 1024x1024 (no detections), 2.8ms
image 6/706 /home/rshah133/bcd/dataset/test/images/03b3656c726cbe0d79c86a25d4296559.jpg: 1024x1024 (no detections), 2.8ms
image 7/706 /home/rshah133/bcd/dataset/test/images/040cc172596bded4092c44094dc33fb7.jpg: 1024x1024 (no detections), 2.8ms
image 8/706 /home/rshah133/bcd/dataset/test/images/042733688eecb852ea06140448a8b1bf.jpg: 1024x1024 1 Mass, 4 Spiculations, 2.8ms
image 9/706 /home/rshah13

In [7]:
# best_model = YOLO('runs/detect/checkpoint/weights/best.pt')
# test_img_path = data_loaded['val']
# results = best_model.predict(source = test_img_path, save = True,  save_txt = True,  conf = 0.10, batch = 16)

Run this cell as it is and take screenshot of the output. This code is for calculating the metrics for test data

u can play with conf keep between 0.05-0.2 (for our use case). lower value means it predicts more but with less accuracy. 

In [8]:
metrics = best_model.val(data = test_path, conf = 0.10)

Ultralytics 8.3.68 🚀 Python-3.11.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81156MiB)


val: Scanning /home/rshah133/bcd/dataset/test/labels... 706 images, 72 backgrounds, 0 corrupt: 100%|██████████| 706/706 [00:00<00:00, 931.48it/s] 

val: New cache created: /home/rshah133/bcd/dataset/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:04<00:00,  9.18it/s]


                   all        706       2476     0.0937     0.0152     0.0516     0.0242
                  Mass        412        459      0.403      0.126      0.238     0.0989
           Spiculation        538       1627      0.294     0.0356      0.159     0.0646
Suspicious Calcification        149        192      0.333    0.00521      0.171      0.103
Architectural Distortion         29         29          0          0          0          0
             Asymmetry         35         35          0          0          0          0
       Focal Asymmetry         80         80          0          0          0          0
       Skin Thickening         13         13          0          0          0          0
      Global Asymmetry          7          7          0          0          0          0
 Suspicious Lymph Node         22         24          0          0          0          0
       Skin Retraction          2          3          0          0          0          0
     Nipple Retra

In [9]:
def get_prediction_dirs():
    base_dir = "runs/detect"
    
    predict_dirs = [d for d in os.listdir(base_dir) if d.startswith('predict')]
    predict_dirs.sort(key=lambda x: int(x.replace('predict', '')) if x != 'predict' else 0)
    
    return f"{base_dir}/{predict_dirs[-1]}/labels"

predictions_dir = get_prediction_dirs()
predictions_dir

'runs/detect/predict/labels'

In [10]:
test_images_dir = data_loaded['val']
test_labels_dir = os.path.join(os.path.dirname(data_loaded['val']), 'labels')

# Create output directory if it doesn't exist
os.makedirs(output_labels_dir, exist_ok=True)
os.makedirs(output_pred_dir, exist_ok=True)

# Function to read YOLO format labels
def read_yolo_labels(label_file):
    labels = []
    with open(label_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts[:5])
            confidence = float(parts[5]) if len(parts) > 5 else None
            labels.append((int(class_id), x_center, y_center, width, height, confidence))
    return labels

# Function to draw bounding boxes on images
def draw_boxes(image, boxes):
    h, w, _ = image.shape
    for box in boxes:
        class_id, x_center, y_center, width, height, confidence = box
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)
        # Draw rectangle
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color_dict[class_id], 2)
    return image

# Loop through each image
for image_file in os.listdir(test_images_dir):
    if image_file.endswith((".jpg", ".png", ".jpeg")):
        base_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(test_images_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading {image_file}")
            continue

        # Read ground truth labels
        ground_truth_file = os.path.join(test_labels_dir, f"{base_name}.txt")
        ground_truth_boxes = read_yolo_labels(ground_truth_file) if os.path.exists(ground_truth_file) else []

        # Read prediction labels
        prediction_file = os.path.join(predictions_dir, f"{base_name}.txt")
        prediction_boxes = read_yolo_labels(prediction_file) if os.path.exists(prediction_file) else []

        # Draw ground truth (green) and predictions (blue)
        image_with_boxes_gt = draw_boxes(image.copy(), ground_truth_boxes)
        image_with_boxes_pt = draw_boxes(image.copy(), prediction_boxes)

        # Save annotated image
        gt_output_path = os.path.join(output_labels_dir, image_file)
        cv2.imwrite(gt_output_path, image_with_boxes_gt)

        pt_output_path = os.path.join(output_pred_dir, image_file)
        cv2.imwrite(pt_output_path, image_with_boxes_pt)

print("Done saving the predictions")

Done saving the predictions


: 

Code to combine images in a single pdf. This code take upto 20mins to execute

In [2]:
output_labels_file_names = sorted(os.listdir(output_labels_dir))
output_pred_file_names = sorted(os.listdir(output_pred_dir))
c = 0
pdf_file_path = "combined_images.pdf"

with PdfPages(pdf_file_path) as pdf:
    for label_file, pred_file in zip(output_labels_file_names, output_pred_file_names):
        label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
        pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))
        
        img = cv2.resize(pred_img, (640, 640))
        rgb_img = img.copy()
        img = np.float32(img) / 255
        
        fig, axs = plt.subplots(1, 2, figsize=(16, 6))
        axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
        axs[0].set_title('Ground Truth')
        axs[0].axis('off')
        axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
        axs[1].set_title('Prediction')
        axs[1].axis('off')
        
        c+=1
        pdf.savefig(fig)
        plt.close(fig)
        print(f"Processed {pred_file}, Image number: {c}")
    
print(f"{c} prediction images saved in {pdf_file_path}")

Processed 002460132586dc0c7b88a59dce6e77bd.jpg and 002460132586dc0c7b88a59dce6e77bd.jpg, Image number: 1
Processed 008bc6050f6d31fc255e5d87bcc87ba2.jpg and 008bc6050f6d31fc255e5d87bcc87ba2.jpg, Image number: 2
Processed 008c66563c73b2f5b8e42915b2cd6af5.jpg and 008c66563c73b2f5b8e42915b2cd6af5.jpg, Image number: 3
Processed 00a6b0d56eb5136c1be2c3d624b04dad.jpg and 00a6b0d56eb5136c1be2c3d624b04dad.jpg, Image number: 4
Processed 01df962b078e38500bf9dd9969a50083.jpg and 01df962b078e38500bf9dd9969a50083.jpg, Image number: 5
Processed 03b3656c726cbe0d79c86a25d4296559.jpg and 03b3656c726cbe0d79c86a25d4296559.jpg, Image number: 6
Processed 040cc172596bded4092c44094dc33fb7.jpg and 040cc172596bded4092c44094dc33fb7.jpg, Image number: 7
Processed 042733688eecb852ea06140448a8b1bf.jpg and 042733688eecb852ea06140448a8b1bf.jpg, Image number: 8
Processed 04ddd99e0fad1aeca36740eaacdf5607.jpg and 04ddd99e0fad1aeca36740eaacdf5607.jpg, Image number: 9
Processed 051525b959eefb14e973acc7ab8fcfe0.jpg and 0515

: 

In [ ]:
# %matplotlib inline
# from yolo_cam.eigen_cam import EigenCAM
# from yolo_cam.utils.image import show_cam_on_image

# output_labels_file_names = sorted(os.listdir(output_labels_dir))
# output_pred_file_names = sorted(os.listdir(output_pred_dir))
# c = 0
# pdf_file_path = "combined_images.pdf"

# # model = YOLO('/home/rshah133/bcd/v8n_1024/runs/detect/checkpoint/weights/best.pt')
# model = model.cpu()
# target_layers = [model.model.model[-4]]
# cam = EigenCAM(model, target_layers, task='od')


# with PdfPages(pdf_file_path) as pdf:
#     for label_file, pred_file in zip(output_labels_file_names, output_pred_file_names):
#         label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
#         pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))
        
#         img = cv2.resize(pred_img, (640, 640))
#         rgb_img = img.copy()
#         img = np.float32(img) / 255
        
#         grayscale_cam = cam(rgb_img)[0, :, :]
        
#         grayscale_cam_resized = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
#         cam_image = show_cam_on_image(img, grayscale_cam_resized, use_rgb=True)
        
#         fig, axs = plt.subplots(1, 3, figsize=(16, 6))
#         axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
#         axs[0].set_title('Ground Truth')
#         axs[0].axis('off')
#         axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
#         axs[1].set_title('Prediction')
#         axs[1].axis('off')
#         axs[2].imshow(cv2.cvtColor(cam_image, cv2.COLOR_BGR2RGB))
#         axs[2].set_title('EigenCAM Explainable AI')
#         axs[2].axis('off')
        
#         c+=1
#         pdf.savefig(fig)
#         plt.close(fig)
    
# print(f"{c} prediction images saved in {pdf_file_path}")

In [ ]:
# c = 0

# output_labels_file_names = sorted(os.listdir(output_labels_dir))
# output_pred_file_names = sorted(os.listdir(output_pred_dir))

# pdf_file_path = "combined_images.pdf"
# with PdfPages(pdf_file_path) as pdf:
#     for label_file, pred_file in zip(output_labels_file_names[:5], output_pred_file_names[:5]):
#         label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
#         pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))

#         fig, axs = plt.subplots(1, 2, figsize=(16, 6))
#         fig.suptitle(f"Image: {label_file}", fontsize=12)
#         axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
#         axs[0].set_title('Ground Truth')
#         axs[0].axis('off')
#         axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
#         axs[1].set_title('Prediction')
#         axs[1].axis('off')
#         plt.tight_layout()
#         c += 1
        
#         pdf.savefig(fig)
#         plt.close(fig)

# print(f"{c} prediction images saved in {pdf_file_path}")